In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q pytorch-lightning

!unzip /content/drive/MyDrive/grass/datasets/train.zip -d /content/train

In [ ]:

import os
from glob import glob
from pathlib import Path

import matplotlib.pyplot as plt
import pytorch_lightning as pl
import torch
from PIL import Image
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.transforms.functional import center_crop
from torchvision.utils import make_grid, save_image
from tqdm.auto import tqdm

import torch.nn.functional as F
from torch.nn import Module, Conv2d
from torch.nn.utils import spectral_norm
from torch.nn.functional import interpolate, relu

import pandas as pd


In [ ]:
config_defaults = {
    'BATCH_SIZE' : 8,
    'IN_CHANNEL' : 7,
    'OUT_CHANNEL' : 3,
    'LOAD_CHECKPOINT' : False,
    'PATH_CONTEXT' : '/content/train/',
    'PATH_DATA' : '/content/drive/MyDrive/grass/datasets/train.zip',
    'PATH_CHECKPOINT' : '/content/drive/MyDrive/grass/experiments/example-sea-creature-3/lightning_logs/version_21/checkpoints/epoch=8-step=4500.ckpt',
    'Lr_gen' : 0.0003,
    'Lr_disc' : 0.00001,
    'MAX_EPOCH' : 10,
    'SAVE_NTH_BATCH' : 50,
    'DATASET_SIZE' : 2000,
    'LAMBDA_RECON' : 100,
    'HALF_SIZE_LOSS_WEIGHT' : 1,
    'DISPLAY_LOSS_N': 100,
    'TRAIN_DISCRIMINATOR': 3,
    'RESULT_PATH': '/content/drive/MyDrive/grass/experiments/example-sea-creature-noise/',
    'INPUT_DIM': 50*30*128,
    'INITIAL_FILTER_SIZE': 128,
    }

CONFIG = config_defaults

In [ ]:
class GrassDataset(Dataset):
  def __init__(self, path, num_items = -1):
    self.df = pd.read_csv(path + "train.csv")
    self.path = path
    self.length = num_items
    if num_items == -1:
      self.length = len(self.df)
    else:
      self.length = min(len(self.df),num_items)
      self.df = self.df.sample(n = self.length, replace=False)
    self.df = self.df.reset_index(drop=True)
    self.df.head()

  def __len__(self):
    return self.length

  def __getitem__(self, idx):
    inputImagePath = self.path + self.df['Input'][idx][2:]
       
    inputImage = Image.open(inputImagePath)
    inputImage = transforms.functional.to_tensor(inputImage)

   
    depthImagePath = self.path + self.df['Depth'][idx][2:]
    depthImage = Image.open(depthImagePath)
    depthImage = transforms.functional.to_tensor(depthImage)

    nImagePath = self.path + self.df['Normal'][idx][2:]
    nImage = Image.open(nImagePath)
    nImage = transforms.functional.to_tensor(nImage)


    realImagePath = self.path + self.df['Output'][idx][2:]
    realImage = Image.open(realImagePath)
    real = transforms.functional.to_tensor(realImage)

    #                                       here
    condition = torch.cat((inputImage, depthImage[0:1, :, :], nImage), 0) # (3+1+3 = 7) x 480 x 800

    return idx, real, condition

class RAMGrassDataset(Dataset): 
#This one loads the dataset onto the RAM, to speed up training speed, especially if you're running a large number of epochs
#Be very careful about how many items you want there to be here
  def __init__(self, path, num_items = -1):
    print("preparing RAM dataset, hopefully this doesn't take very long")
    self.df = pd.read_csv(path + "train.csv")
    self.path = path
    self.length = num_items
    if num_items == -1:
      self.length = len(self.df)
    else:
      self.length = min(len(self.df),num_items)
      self.df = self.df.sample(n = self.length, replace=False)
    self.df = self.df.reset_index(drop=True)


    self.real = []
    self.condition = []
    print("prepared all variables, adding data to RAM")
    for i in range(self.length):
      idx, real, condition = self.getitem(i)
      self.real.append(real)
      self.condition.append(condition)
    print("dataset added to RAM")

  def __len__(self):
    return self.length

  def __getitem__(self, idx):
    return self.real[idx], self.condition[idx]

  def getitem(self, idx):
    inputImagePath = self.path + self.df['Input'][idx][2:]
       
    inputImage = Image.open(inputImagePath)
    inputImage = transforms.functional.to_tensor(inputImage)

    
    depthImagePath = self.path + self.df['Depth'][idx][2:]
    depthImage = Image.open(depthImagePath)
    depthImage = transforms.functional.to_tensor(depthImage)

    nImagePath = self.path + self.df['Normal'][idx][2:]
    nImage = Image.open(nImagePath)
    nImage = transforms.functional.to_tensor(nImage)


    realImagePath = self.path + self.df['Output'][idx][2:]
    realImage = Image.open(realImagePath)
    real = transforms.functional.to_tensor(realImage)

    #                                       here
    condition = torch.cat((inputImage, depthImage[0:1, :, :], nImage), 0) # (3+1+3 = 7) x 480 x 800

    return idx, real, condition


In [ ]:
class DownSampleConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel=4, strides=2, padding=1, activation=True, batchnorm=True):

        super().__init__()
        self.activation = activation
        self.batchnorm = batchnorm

        self.conv = nn.Conv2d(in_channels, out_channels, kernel, strides, padding)

        if batchnorm:
            self.bn = nn.BatchNorm2d(out_channels)
            #self.bn = nn.GroupNorm(1, out_channels)

        if activation:
            self.act = nn.LeakyReLU(0.2)

    def forward(self, x):
        x = self.conv(x)
        if self.batchnorm:
            x = self.bn(x)
        if self.activation:
            x = self.act(x)
        return x

class UpSampleConv(nn.Module):

    def __init__(
        self,
        in_channels,
        out_channels,
        kernel=4,
        strides=2,
        padding=1,
        activation=True,
        batchnorm=True,
        dropout=False
    ):
        super().__init__()
        self.activation = activation
        self.batchnorm = batchnorm
        self.dropout = dropout

        self.deconv = nn.ConvTranspose2d(in_channels, out_channels, kernel, strides, padding)

        if batchnorm:
            self.bn = nn.BatchNorm2d(out_channels)

        if activation:
            self.act = nn.ReLU(True)

        if dropout:
            self.drop = nn.Dropout2d(0.5)

    def forward(self, x):
        x = self.deconv(x)
        if self.batchnorm:
            x = self.bn(x)

        if self.dropout:
            x = self.drop(x)
        return x

In [ ]:
# class ResnetBlock(nn.Module):
#     def __init__(self, dim, padding_type, norm_layer=nn.BatchNorm2d, activation=nn.ReLU(True), use_dropout=False):
#         super(ResnetBlock, self).__init__()
#         self.build_conv_block(dim, padding_type, norm_layer, activation, use_dropout)

#     def build_conv_block(self, dim, padding_type, norm_layer, activation, use_dropout):
#         self.activation = activation;
#         self.norm_layer = norm_layer;
#         self.dropout = use_dropout;

#         if(norm_layer):
#             self.nl = nn.BatchNorm2d(dim)
#             self.nl2 = nn.BatchNorm2d(dim)

          
#         if activation:
#             self.act = nn.ReLU(True)
#             self.act2 = nn.ReLU(True)
          
#         if use_dropout:
#             self.dp = nn.Dropout(0.5)

#         self.conv_1 = nn.Conv2d(dim, dim, kernel_size=3, padding=1)


#         self.conv_2 = nn.Conv2d(dim, dim, kernel_size=3, padding=1)


#     def forward(self, x):
        
#         y = self.conv_1(x)
#         y = self.nl(y)
#         y = self.act(y)

#         #y = self.dp(y) Regularisation

#         y = self.conv_2(y)
#         y = self.nl2(y)
#         y = self.act2(y)

#         return x + y

In [ ]:
# class RobGenerator(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super().__init__()
       
#         self.layers = [
#             DownSampleConv(in_channels, 64, batchnorm=False),  # bs x 64 x 240 x 400
#             DownSampleConv(64, 128),  # bs x 128 x 120 x 200
#             ResnetBlock(128, padding_type = 'reflect', use_dropout=True), # bs x 128 x 120 x 200
#             ResnetBlock(128, padding_type = 'reflect', use_dropout=True), # bs x 128 x 120 x 200
#             ResnetBlock(128, padding_type = 'reflect', use_dropout=True), # bs x 128 x 120 x 200
#             UpSampleConv(128, 128),  # bs x 128 x 240 x 400
#             ResnetBlock(128, padding_type = 'reflect', use_dropout=True), # bs x 128 x 240 x 400
#             ResnetBlock(128, padding_type = 'reflect', use_dropout=True), # bs x 128 x 240 x 400
#             nn.ConvTranspose2d(128, out_channels, kernel_size=4, stride=2, padding=1), # bs x 3 x 480 x 800
#         ]

#         self.tanh = nn.Tanh()
#         self.layers = nn.ModuleList(self.layers)

#     def forward(self, x):
#         for layer in self.layers:
#             x = layer(x)
#         return self.tanh(x)

In [ ]:
class SPADE(Module):
    def __init__(self, k):
        super().__init__()
        num_filters = 64
        kernel_size = 3
        self.conv = spectral_norm(Conv2d(CONFIG["IN_CHANNEL"], num_filters, kernel_size=(kernel_size, kernel_size), padding=1)) #made changes on inputChannel
        self.conv_gamma = spectral_norm(Conv2d(num_filters, k, kernel_size=(kernel_size, kernel_size), padding=1))
        self.conv_beta = spectral_norm(Conv2d(num_filters, k, kernel_size=(kernel_size, kernel_size), padding=1))

    def forward(self, x, seg):
        #x = (b, 128, h, w), seg = (b, 4, h,w )
        # print('In SPADE')
        # seg_copy = self.convseg(seg) #seg = (b, 128, h,w)
        N, C, H, W = x.size()
        sum_channel = torch.sum(x.reshape(N, C, H*W), dim=-1)

        mean = sum_channel / (N*H*W)

        std = torch.sqrt((sum_channel**2 - mean**2) / (N*H*W))
        mean = torch.unsqueeze(torch.unsqueeze(mean, -1), -1)
        std = torch.unsqueeze(torch.unsqueeze(std, -1), -1)
        x = (x - mean) / std

        # seg = interpolate(seg, size=(H,W), mode='nearest')
        seg_copy = relu(self.conv(seg))  #------------------------------->CPU and !CUDA
        # print(f'seg_copy: {seg_copy.shape}')

        seg_gamma = self.conv_gamma(seg_copy)
        # print(f'seg_gamma: {seg_gamma.shape}')

        seg_beta = self.conv_beta(seg_copy)
        # print(f'seg_beta: {seg_beta.shape}')

        #torch.matmul performs matrix multiplication so for the given equal sized vectors need to make size of seg_gamma such that seg_gamma.size(3)=x.size(2)
        #so taking the difference between those two those indices concatinating the difference to x
        #x_copy = torch.cat((x, torch.zeros(x.size(0), x.size(1), seg_gamma.size(3)-x.size(2), x.size(3)).to(device)), dim = 2)
        # x = (torch.matmul(seg_gamma, x_copy) + seg_beta)

        x = torch.mul(seg_gamma, x) + seg_beta
        # print(f'exit: {x.shape}')
        return x

class SPADEResBlk(Module):
    def __init__(self, k, skip=False):
        super().__init__()
        kernel_size = 3
        self.skip = skip
        
        if self.skip:
            self.spade1 = SPADE(2*k)
            self.conv1 = Conv2d(2*k, k, kernel_size=(kernel_size, kernel_size), padding=1, bias=False)
            self.spade_skip = SPADE(2*k)
            self.conv_skip = Conv2d(2*k, k, kernel_size=(kernel_size, kernel_size), padding=1, bias=False)
        else:
            self.spade1 = SPADE(k)
            self.conv1 = Conv2d(k, k, kernel_size=(kernel_size, kernel_size), padding=1, bias=False)

        self.spade2 = SPADE(k)
        self.conv2 = Conv2d(k, k, kernel_size=(kernel_size, kernel_size), padding=1, bias=False)
    
    def forward(self, x, seg):
        # print(f'In SpadeResBlk')
        x_skip = x
        # print(f'before spade1: x {x.shape} seg: {seg.shape} ')
        x = relu(self.spade1(x, seg)) #b, 128, 30, 50
        # print(f'x: {x.shape}')
        # print(f'After spade1: {x.shape}')
        x = self.conv1(x) #b, 128, 30, 50

        # print(f'After conv1: {x.shape}')
        x = relu(self.spade2(x, seg) ) #b, 128, 30,50  
        x = self.conv2(x) #b, 128, 30, 50

        if self.skip:
            x_skip = relu(self.spade_skip(x_skip, seg))
            x_skip = self.conv_skip(x_skip)
        # print(f'After spade1: {(x_skip + x).shape}')
        return x_skip + x

class SPADEGenerator(nn.Module):
    def __init__(self, inchannels, outchannels): #---------_Remember inchannels
        super().__init__()
 
        self.ln = nn.Linear(CONFIG['INITIAL_FILTER_SIZE'], CONFIG['INPUT_DIM'])

        self.fc = nn.Conv2d(inchannels, 128, kernel_size = (3,3), padding = 1)
        self.spade_resblk1 = SPADEResBlk(128)
        self.spade_resblk2 = SPADEResBlk(128)
        self.spade_resblk3 = SPADEResBlk(64, skip=True)
        self.spade_resblk4 = SPADEResBlk(32, skip=True)
        self.conv = nn.utils.spectral_norm(nn.Conv2d(32, outchannels, kernel_size=(3,3), padding=1))
        self.convTrans128 = nn.ConvTranspose2d(128, 128, kernel_size = 2, stride = 2)
        self.convTrans64 = nn.ConvTranspose2d(64, 64, kernel_size = 2, stride = 2)
        self.convTrans32 = nn.ConvTranspose2d(32, 32, kernel_size = 2, stride = 2)
        self.noise = torch.normal(0, 1, (CONFIG['BATCH_SIZE'], CONFIG['INITIAL_FILTER_SIZE'])).to(device)


    
    def forward(self, seg):
  
        x = self.ln(self.noise)
        #print(f'After linear {x.shape}')
        x = x.view(CONFIG['BATCH_SIZE'], -1, 30, 50) #change 4 4 such that -1 = 128 If change this also change hidden layer size with x*y
        #print(f'After view {x.shape}')
        #x.shape = 8,4,15,25
        m4 = F.interpolate(seg, scale_factor = 0.5, mode = "bicubic") #b, 4, 240, 400
        m3 = F.interpolate(m4, scale_factor = 0.5, mode = "bicubic") #b, 4, 120, 200
        m2 = F.interpolate(m3, scale_factor = 0.5, mode = "bicubic") #b, 4, 60, 100
        m1 = F.interpolate(m2, scale_factor = 0.5, mode = "bicubic") #b, 4, 30, 50 

        # print(f'm1: {m1.shape}')
        #x = self.fc(m2) #b, 128, 30, 50
        x = self.spade_resblk1(x, m1) #b, 128, 30, 50
        x= self.convTrans128(x)
        #x = F.interpolate(x, scale_factor = 2, mode='bicubic') #b, 128, 60, 100
    
        #print(f'interpolate1: {x.shape}')

        x = self.spade_resblk2(x, m2) #b, 128, 60, 100
        #print(f'resblk2: {x.shape}')
        x= self.convTrans128(x)
        #x = F.interpolate(x, scale_factor = 2, mode='bicubic')#b, 128, 120, 200
        #print(f'interpolate 2: {x.shape}')

        x = self.spade_resblk3(x, m3)  #b, 64, 120,200
        #print(f'resblk3: {x.shape}')
        x= self.convTrans64(x)

       # x = F.interpolate(x,scale_factor = 2, mode='bicubic')#b, 64, 240, 400 
       # print(f'interpolate3: {x.shape}')
        
        x = self.spade_resblk4(x, m4)  #b, 32, 240, 400
        # print(f'resblk4: {x.shape}')
        x= self.convTrans32(x) #b,32,480,800

       # x = F.interpolate(x, scale_factor = 2, mode='bicubic')#b, 32, 480, 800
        # print(f'interpolate 4: {x.shape}')

        x = F.tanh(self.conv(x)) #b, 3, 480, 800
        # print(f'tanh: {x.shape}')
        
        return x

In [ ]:
# import torch
# l = torch.nn.Linear(Initial_filters, Height*Weight*Initial_Filters)
# x = torch.normal(0, 1, (batch_size, Initial_Filters))
# x = l(x)
# x = x.view(8, -1, 30, 50) #change 4 4 such that -1 = 128 If change this also change hidden layer size with x*y
# print(x.shape)

In [ ]:
class PatchGAN(nn.Module):
    def __init__(self, input_channels):
        super().__init__()
        self.d1 = DownSampleConv(input_channels, 32, batchnorm=False)
        self.d2 = DownSampleConv(32, 64)
        self.d3 = DownSampleConv(64, 64)
        self.d4 = DownSampleConv(64, 64)
        self.final = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x, y):
        x = torch.cat([x, y], axis=1)
        x0 = self.d1(x)
        x1 = self.d2(x0)
        x2 = self.d3(x1)
        x3 = self.d4(x2)
        xn = self.final(x3)
        return xn

class MultilevelPatchGAN(nn.Module):
    def __init__(self, input_channels):
        super().__init__()
        self.d1 = DownSampleConv(input_channels, 32, batchnorm=False)
        self.d2 = DownSampleConv(32, 64)
        self.d3 = DownSampleConv(64, 64)
        self.d4 = DownSampleConv(64, 64)
        self.final = nn.Conv2d(64, 1, kernel_size=1)

        self.dd1 = DownSampleConv(input_channels, 32, batchnorm=False)
        self.dd2 = DownSampleConv(32, 64)
        self.dd3 = DownSampleConv(64, 64)
        self.dfinal = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x, y):
        x = torch.cat([x, y], axis=1)
        xx = torch.nn.functional.interpolate(x, scale_factor=0.5, mode="bicubic")

        x0 = self.d1(x)
        x1 = self.d2(x0)
        x2 = self.d3(x1)
        x3 = self.d4(x2)
        xn = self.final(x3)

        xx0 = self.dd1(xx)
        xx1 = self.dd2(xx0)
        xx2 = self.dd3(xx1)
        xxn = self.dfinal(xx2)
        return xn, xxn

In [ ]:
def _weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)
        
def display_progress(cond, fake, real, figsize=(10,5)):
    cond = cond.detach().cpu().permute(1, 2, 0)
    fake = fake.detach().cpu().permute(1, 2, 0)
    real = real.detach().cpu().permute(1, 2, 0)
    
    fig, ax = plt.subplots(1, 3, figsize=figsize)
    #print(cond.shape)
    ax[0].imshow(cond[:,:,0:3])
    ax[2].imshow(fake)
    ax[1].imshow(real)
    plt.show()


def draw_result(lst_itr, lst_err1, lst_err2, label1, label2):
  plt.plot(lst_itr, lst_err1, '-b', label = label1)
  plt.plot(lst_itr, lst_err2, '-r', label = label2)
  plt.xlabel('iteration')
  plt.show()

In [ ]:
class GAN(pl.LightningModule):

    def __init__(self, in_channels, out_channels, learning_rate=0.0002, lambda_recon=200, next_save_idx = 0):

        super().__init__()
        self.save_hyperparameters()


        self.gen = SPADEGenerator(in_channels, out_channels)
        self.patch_gan = PatchGAN(in_channels + out_channels)

        # intializing weights
        self.gen = self.gen.apply(_weights_init)
        self.patch_gan = self.patch_gan.apply(_weights_init)

        self.adversarial_criterion = nn.BCEWithLogitsLoss()
        self.recon_criterion = nn.L1Loss()
        self.trainidx = next_save_idx

        self.lambda_recon = CONFIG["LAMBDA_RECON"]

        self.recon_lossl = []
        self.adverserial_lossl = []
        self.itrg = []
        self.itrd = []
        self.real_lossl = []
        self.fake_lossl = []

    def _gen_step(self, real_images, conditioned_images):
        fake_images = self.gen(conditioned_images)
        disc_logits = self.patch_gan(fake_images, conditioned_images)
        adversarial_loss = self.adversarial_criterion(disc_logits, torch.ones_like(disc_logits))

        recon_loss = self.recon_criterion(fake_images, real_images)
        self.adverserial_lossl.append(adversarial_loss.cpu().data.numpy())
        self.recon_lossl.append(recon_loss.cpu().data.numpy())
        self.itrg.append(len(self.adverserial_lossl))
        return self.lambda_recon *adversarial_loss +  recon_loss
        
    def _disc_step(self, real_images, conditioned_images):
        fake_images = self.gen(conditioned_images).detach()

        fake_logits = self.patch_gan(fake_images, conditioned_images)
        real_logits = self.patch_gan(real_images, conditioned_images)

        fake_loss = self.adversarial_criterion(fake_logits, torch.zeros_like(fake_logits))
        real_loss = self.adversarial_criterion(real_logits, torch.ones_like(real_logits))
        self.fake_lossl.append(fake_loss.cpu().data.numpy())
        self.real_lossl.append(real_loss.cpu().data.numpy())
        self.itrd.append(len(self.fake_lossl))
        return (real_loss + fake_loss) / 2

    def configure_optimizers(self):
        gen_opt = torch.optim.Adam(self.gen.parameters(), lr=CONFIG['Lr_gen'])
        disc_opt = torch.optim.Adam(self.patch_gan.parameters(), lr=CONFIG['Lr_disc'])
        return disc_opt, gen_opt

    def training_step(self, batch, batch_idx, optimizer_idx):
        idx, real, condition = batch

        loss = None

        if optimizer_idx == 1:
            loss = self._gen_step(real, condition)

            self.log('Generator Loss', loss)
        
        elif(batch_idx%CONFIG['TRAIN_DISCRIMINATOR']==0 and optimizer_idx ==0 ):

            loss = self._disc_step(real, condition)

            self.log('PatchGAN Loss', loss)
        

        if batch_idx% CONFIG['SAVE_NTH_BATCH']==0and optimizer_idx==1:
            print(f'batch size {batch_idx}')
            fake = self.gen(condition).detach()
            display_progress(condition[0], fake[0], real[0])
            for i in range(1):
              index = int(idx[i])+1
              path = CONFIG['RESULT_PATH'] + str(self.trainidx) + "_" + str(index) + ".png"
              self.trainidx+=1
              save_image(fake[i], path)
              # draw_result(self.itrd, self.fake_lossl, self.real_lossl, label1 = 'Fake Loss', label2 = 'Real Loss') 
              # draw_result(self.itrg, self.recon_lossl, self.adverserial_lossl, label1 = 'Recon Loss', label2 = 'Adverserial Loss') 

              # print(f'batch: {batch_idx}/{ CONFIG['DATASET_SIZE']/CONFIG['BATCH_SIZE']}')

        if batch_idx%CONFIG['DISPLAY_LOSS_N'] ==0 and batch_idx != 0:
          draw_result(self.itrd, self.fake_lossl, self.real_lossl, label1 = 'Fake Loss', label2 = 'Real Loss') 
          draw_result(self.itrg, self.recon_lossl, self.adverserial_lossl, label1 = 'Recon Loss', label2 = 'Adverserial Loss') 


        return loss

    def forward(self, x, y, z):
        x = torch.concat((x,y,z), axis=1)
        return self.gen(x)


In [ ]:
#load dataset
dataset = GrassDataset(CONFIG['PATH_CONTEXT'], CONFIG['DATASET_SIZE'])
dataloader = DataLoader(dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True)

In [ ]:
xadversarial_loss = nn.BCEWithLogitsLoss()
reconstruction_loss = nn.L1Loss()

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

if CONFIG['LOAD_CHECKPOINT']:
  model = GAN.load_from_checkpoint(CONFIG['PATH_CHECKPOINT'])
else:
  model = GAN(CONFIG['IN_CHANNEL'], CONFIG['OUT_CHANNEL'])

trainer = pl.Trainer(max_epochs=CONFIG['MAX_EPOCH'], gpus=1, default_root_dir = CONFIG['RESULT_PATH'])
trainer.fit(model, dataloader)

TypeError: ignored

### from 60*100 size
1. added recon factor in adverserial

2. decreased learning rate of discriminator
3. Performed element wise multiplication
4. Changed the number of times the discriminator trains

### in 30 * 50
1. Results are too smooth

to-Do:
add update_learning_rate



version: 21

## After training:
1. the losses remain same after nth epoch
2. Recon loss : ~0.1
3. Real loss: ~0.67
4. fake loss: ~0.71
5. Adverserial: ~0.7

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# import torch
# from PIL import Image
# from torchvision.transforms import transforms

# image = Image.open("/content/train/Input/0_input.jpg")
# image.show()
# T = transforms.ToTensor()
# reI = T(image)
# reI = torch.unsqueeze(reI, dim = 0)
# print(reI.shape)
# rescaled1 = torch.nn.functional.interpolate(reI, scale_factor = 0.5, mode = "bicubic") # 200
# rescaled2 = torch.nn.functional.interpolate(rescaled1, scale_factor = 0.5, mode = "bicubic")#100
# rescaled3 = torch.nn.functional.interpolate(rescaled2, scale_factor = 0.5, mode = "bicubic")#50

# rescaled1 = torch.squeeze(rescaled1)
# rescaled2 = torch.squeeze(rescaled2)
# rescaled3 = torch.squeeze(rescaled3)

# I = transforms.ToPILImage()
# en1 = I(rescaled1)
# en2 = I(rescaled2)
# en3 = I(rescaled3)
# en1.show()
# en2.show()
# en3.show()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

EXPERIMENT INFORMATION


Verson 13: Even after 9th epoch the results seems to be consistent. The model is not learning anything. It is learning about the lightning, structures and colors very well but it's not getting the textures right(or at all). Here we started training it from 60,100 size. 
